In [124]:
#!touch 0-inception_block.py | chmod +x *.py
!pwd

/home/khouloud/hb/holbertonschool-machine_learning/supervised_learning/0x08-deep_cnns


In [123]:
#!/usr/bin/env python3

import tensorflow.keras as K
#inception_block = __import__('0-inception_block').inception_block
def inception_block(A_prev, filters):
    """ doc """
    MaxPooling2D = K.layers.MaxPooling2D
    Conv2D = K.layers.Conv2D
    Concatenate = K.layers.Concatenate()
    
    F1, F3R, F3, F5R, F5, FPP = filters
    
    layer1x1_0 = Conv2D(F1, 1, activation='relu')(A_prev) #
    
    layer1x1_1 = Conv2D(F3R, 1, padding='same', activation='relu')(A_prev)
    
    layer3x3 = Conv2D(F3, 3, padding='same', activation='relu')(layer1x1_1) #
    
    layer1x1_2 = Conv2D(F5R, 1, padding='same', activation='relu')(A_prev)
    
    layer5x5 = Conv2D(F5, 5, padding='same',  activation='relu')(layer1x1_2) #
    
    layerMax = MaxPooling2D(1)(A_prev)
    
    layer1x1_3 = Conv2D(FPP, 1, padding='same', activation='relu')(layerMax) #
    
    layer_out = Concatenate([layer1x1_0, layer3x3, layer5x5, layer1x1_3])
    return layer_out
if __name__ == '__main__':
    X = K.Input(shape=(224, 224, 3))
    Y = inception_block(X, [64, 96, 128, 16, 32, 32])
    model = K.models.Model(inputs=X, outputs=Y)
    model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_66 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_370 (Conv2D)             (None, 224, 224, 96) 384         input_66[0][0]                   
__________________________________________________________________________________________________
conv2d_372 (Conv2D)             (None, 224, 224, 16) 64          input_66[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_63 (MaxPooling2D) (None, 224, 224, 3)  0           input_66[0][0]                   
__________________________________________________________________________________________________
conv2d_369

In [ ]:
 [(None, 224, 224, 64), (None, 222, 222, 128), (None, 220, 220, 32), (None, 224, 224, 32)]